In [1]:
# Create MNIST data arrays
%run ./generate_mnist_dataset.ipynb

  3%|▎         | 2035/60000 [00:00<00:02, 20344.93it/s]

Preprocessing MNIST training data...


  4%|▍         | 380/10000 [00:00<00:02, 3799.14it/s]

Preprocessing MNIST training data...


100%|██████████| 10000/10000 [00:00<00:00, 15296.56it/s]


In [2]:
# Load and prepare the data

import torch
import os
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm
from training_data import DataCollection
from PIL import Image
from matplotlib import pyplot as plt

def print_data_infos(data_train, data_test):
    print("Train data length: {0}".format(len(data_train.data)))
    print("Test data length: {0}".format(len(data_test.data)))
    print("Img Shape: {0}".format(data_train.data[0].shape))
    print("Number of Labels: {0}".format(data_train.no_labels))
    
data_all_train = DataCollection()
data_all_test = DataCollection(train=False)


print_data_infos(data_all_train, data_all_test)


  0%|          | 0/10000 [00:00<?, ?it/s]

No training data for ). Skipping


100%|██████████| 10000/10000 [00:00<00:00, 341580.73it/s]


No training data for ). Skipping
Train data length: 71969
Test data length: 14427
Img Shape: torch.Size([1, 32, 32])
Number of Labels: 15


In [4]:
# Declare the network and some utilities

from torchvision import models
from torch.nn import Conv2d


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def train(train_loader, test_loader, model_name, print_step, num_classes=15, epochs=15):
    model = models.alexnet(num_classes=num_classes)
    model.features[0] = Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    if torch.cuda.is_available():
        model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.8, 0.99), weight_decay=0.001)
    criterion = nn.CrossEntropyLoss()
    for epoch in range(epochs):
        print("Epoch {0}".format(epoch))
        for step, [x_train, y_train] in enumerate(tqdm(train_loader)):
            if torch.cuda.is_available():
                 x_train, y_train = x_train.to(device), y_train.to(device)
            optimizer.zero_grad()
            train_pred = model(x_train)
            loss = criterion(train_pred, y_train)
            loss.backward()
            optimizer.step()
            if step % print_step == 0:
                print('Loss: {}'.format(loss))
        
        acc = calc_accuracy(model, test_loader)
        print("Accuracy: {0}".format(acc))
        if acc > 98:
            torch.save(model.state_dict(), '{0}-{1}.ckpt'.format(model_name,acc))
    print("Accuracy: {0}".format(acc))
    torch.save(model.state_dict(), '{0}.ckpt'.format(model_name))

def calc_accuracy(model, test_loader):
    accuracies = []
    for idx, [x_test, y_test] in enumerate(tqdm(test_loader)):
        if torch.cuda.is_available():
            x_test, y_test = x_test.to(device), y_test.to(device)
        test_pred = model(x_test)
        accuracy = 100 * torch.mean((torch.argmax(test_pred, dim=1) == y_test).float())
        accuracies.append(accuracy.item() if torch.cuda.is_available() else accuracy)
    return np.mean(accuracies)  

train_all_loader = DataLoader(data_all_train, batch_size=16, shuffle=True)
test_all_loader = DataLoader(data_all_test, batch_size=16, shuffle=False)



In [ ]:
train(train_all_loader, test_all_loader, 'model-all-symbols', 500)

  0%|          | 0/4499 [00:00<?, ?it/s]

Epoch 0


  0%|          | 1/4499 [00:00<50:49,  1.48it/s]

Loss: 2.7100908756256104


 11%|█         | 501/4499 [06:30<41:07,  1.62it/s]  

...

 33%|███▎      | 1501/4499 [23:52<51:24,  1.03s/it]  

Loss: 0.5153787136077881


 44%|████▍     | 2001/4499 [32:48<40:14,  1.03it/s]  

Loss: 0.1105327233672142


 45%|████▍     | 2018/4499 [33:04<39:15,  1.05it/s]

In [ ]:
train_no_strokes = DataCollection(own_path='digits-plus-brackets', no_strokes=True)
test_no_strokes = DataCollection(own_path='digits-plus-brackets', train=False, no_strokes=True)
print_data_infos(train_no_strokes, test_no_strokes)

train_no_strokes_loader = DataLoader(train_no_strokes, batch_size=16, shuffle=True)
test_no_strokes_loader = DataLoader(test_no_strokes, batch_size=16, shuffle=False)

train(train_no_strokes_loader, test_no_strokes_loader, 'model-no_strokes', 500, num_classes=13)